In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from glob import glob
import os

# Reading Data

In [2]:
# Define the relative path pattern for .edf files in all subdirectories of the Data directory
path_pattern = os.path.join('..', '..', 'CleanedEpochs', 'Recording', 'Subj*', '*.fif')
# Use glob to get all matching files
epoch_files = glob(path_pattern, recursive=True)

# Print the list of .fif files
for e in epoch_files:
    print(e)

..\..\CleanedEpochs\Recording\Subj1\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj1\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj10\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj10\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj11\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj11\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj12\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj12\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj13\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj13\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj14\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj14\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj15\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj15\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj16\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj16\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj2\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj2\Slow-epo.fif
..\..\CleanedEpochs\Recording\Subj3\Fast-epo.fif
..\..\CleanedEpochs\Recording\Subj3\Slow-epo.fif
..\..\

In [3]:
md = pd.read_csv('../../recording_metadata.csv')
epochs = [mne.read_epochs(f) for f in epoch_files]

Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Recording\Subj1\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
555 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Recording\Subj1\Slow-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
463 matching events found
No baseline correction applied
0 projection items activated
Reading e:\Campus\RM\FinalProject\Codes\FeatureExtraction\..\..\CleanedEpochs\Recording\Subj10\Fast-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    1992.19 ms
        0 CTF compensation matrices available
Not setting metadata
464 matching events found
No baseline correction applied
0 projection item

# Feature Extraction using PSD

In [4]:
res_df = pd.DataFrame()

In [5]:
FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 50),
}

# Define brain regions and their corresponding channel names
BRAIN_REGIONS = {
    "frontal": ["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"],
    "temporal": ["T7", "T8"],
    "parietal": ["P7", "P8"],
    "occipital": ["O1", "O2"],
}

# Get the channel names from the first epochs object
channels = epochs[0].ch_names

# Extract indices for each brain region
region_indices = {
    region: [channels.index(ch) for ch in ch_names if ch in channels]
    for region, ch_names in BRAIN_REGIONS.items()
}

In [6]:
# Define the function to compute power bands
from mne.time_frequency import psd_array_welch
def get_power_band(epochs, region_indices):
    """
    Input: epochs object
    Output: averaged power band for each brain region
    Output shape: (5, 4) -> 5 frequency bands and 4 brain regions
    """
    sfreq = epochs.info['sfreq']  # Sampling frequency
    epoch_length = epochs.times[-1] - epochs.times[0]  # Duration of each epoch in seconds

    # Set `n_per_seg` to match the duration of each epoch
    n_per_seg = int(sfreq * epoch_length)

    # Set `n_fft` to match `n_per_seg` for computational efficiency
    n_fft = n_per_seg

    # Compute PSD using Welch's method
    psds, freqs = psd_array_welch(epochs.get_data(copy=False), sfreq=sfreq, n_fft=n_fft, n_per_seg=n_per_seg, n_overlap=n_per_seg // 2)
    psds *= 1e12  # Convert to uV^2/Hz
    psds = 10 * np.log10(psds)  # Convert to dB

    # Average over epochs
    psds = psds.mean(axis=0)

    psds_per_region = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        psds_per_region.append(
            [psds_band[region_indices[region]].mean() for region in BRAIN_REGIONS]
        )

    return np.array(psds_per_region)

In [7]:
# Compute power bands and add to DataFrame
for i, epoch in enumerate(epochs):
    psds_per_region = get_power_band(epoch, region_indices)

    # Flatten the power band data
    flattened_data = psds_per_region.flatten()

    # Create column names for the flattened data
    column_names = [
        f"{freq_band}_{region}"
        for freq_band in FREQ_BANDS.keys()
        for region in BRAIN_REGIONS.keys()
    ]

    # Add the flattened data to the DataFrame
    for j, col_name in enumerate(column_names):
        res_df.loc[i, col_name] = flattened_data[j]

Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective window size : 1.992 (s)
Effective wind

In [8]:
res_df

,delta_frontal,delta_temporal,delta_parietal,delta_occipital,theta_frontal,theta_temporal,theta_parietal,theta_occipital,alpha_frontal,alpha_temporal,alpha_parietal,alpha_occipital,beta_frontal,beta_temporal,beta_parietal,beta_occipital,gamma_frontal,gamma_temporal,gamma_parietal,gamma_occipital
0,12.292051,11.329002,11.674087,12.922914,4.066946,2.140754,2.374543,6.175590,-0.143743,-2.428043,-1.777107,2.561042,-2.857164,-6.332464,-5.205582,-1.543010,-9.130058,-12.194344,-10.917182,-8.825350
1,7.309125,10.061056,7.546369,8.974584,2.743722,3.385166,2.630239,4.412384,-0.067064,0.978659,1.734728,2.981149,-4.058494,-3.964753,-3.800120,-2.711307,-10.772057,-10.513060,-11.146979,-10.671033
2,8.337568,8.914537,9.490035,10.030883,2.739611,2.365667,2.871901,4.497629,-1.627847,-2.073946,-1.762912,1.356445,-4.567448,-4.300117,-5.263244,-1.922795,-10.812742,-9.931322,-11.143240,-7.918015
3,7.505322,9.431755,7.438969,8.039955,1.967647,3.041604,1.068027,3.935161,-1.965790,-0.752261,-2.631966,1.068902,-5.264003,-3.890478,-6.021118,-2.424133,-11.880094,-10.301446,-12.248450,-9.752502
4,6.168545,4.318363,4.765268,4.938281,-0.726990,-1.442261,-1.297068,-0.501570,-3.054368,-0.743400,-2.152041,-2.477578,-4.594170,-1.222540,-3.905687,-5.638742,-10.408105,-6.898158,-10.078255,-12.389091
5,10.093726,8.243090,6.107126,5.765474,2.355747,0.517643,-0.756693,-0.155384,-1.170363,-1.671311,-3.750931,-3.282617,-5.260625,-5.234498,-6.374481,-6.132446,-11.944826,-11.472905,-12.833719,-13.163993
6,7.044158,7.901314,7.905463,6.269357,0.885509,0.825187,0.535263,1.098387,-2.114427,-1.731449,-2.681007,-1.185795,-5.317235,-5.475875,-7.103579,-5.888525,-11.506342,-11.732700,-13.100827,-11.956819
7,10.214173,10.329621,12.092609,10.976450,2.779489,2.442172,3.283131,2.786042,-0.578067,-0.031354,0.222113,-0.038397,-4.437386,-2.259175,-5.185475,-5.059013,-10.787386,-8.483997,-12.264738,-11.811599
8,6.544997,8.276109,7.662911,8.180544,0.662702,1.733939,1.328708,2.936749,-2.282989,-1.721989,-1.756773,0.740675,-3.124516,-1.732051,-3.489612,-2.260007,-9.459535,-7.635724,-9.878078,-8.929192
9,8.808022,11.925279,9.622309,8.013964,1.836871,3.409111,1.723030,2.690417,-1.744148,-0.834391,-2.001507,0.141023,-5.501565,-2.953962,-4.721544,-3.728198,-12.491368,-10.553645,-11.969222,-11.398061


In [9]:
# Concat with subject_id and pace
res_df.to_csv('../../recording_band_powers.csv', index=False)